In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").appName("dupli_ex_1").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/26 11:54:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df1 = spark.read.csv('DataSets/dupli.csv', header=True,sep='|')
df1.show()

+-----+---+
| Name|Age|
+-----+---+
|  Nik| 26|
|  Akh| 31|
| Sony| 35|
|  Nik| 26|
|  Akh| 31|
| Sony| 35|
|  Akh| 31|
| Sony| 35|
| Demo| 21|
|Nibba| 16|
|Nibbi| 17|
+-----+---+



In [4]:
df1.dropDuplicates().show() #It takes columns list also

+-----+---+
| Name|Age|
+-----+---+
|  Akh| 31|
|Nibba| 16|
| Sony| 35|
|Nibbi| 17|
| Demo| 21|
|  Nik| 26|
+-----+---+



In [5]:
df1.distinct().show() #It doesn't take any argument

+-----+---+
| Name|Age|
+-----+---+
|  Akh| 31|
|Nibba| 16|
| Sony| 35|
|Nibbi| 17|
| Demo| 21|
|  Nik| 26|
+-----+---+



In [6]:
df1.columns

['Name', 'Age']

GroupBy:

In [8]:
df1.groupBy(df1.columns).count().show()

+-----+---+-----+
| Name|Age|count|
+-----+---+-----+
|  Akh| 31|    3|
|Nibba| 16|    1|
| Sony| 35|    3|
|Nibbi| 17|    1|
| Demo| 21|    1|
|  Nik| 26|    2|
+-----+---+-----+



In [9]:
df1.groupBy(df1.columns).count().where( "count > 1" ).show()

+----+---+-----+
|Name|Age|count|
+----+---+-----+
| Akh| 31|    3|
|Sony| 35|    3|
| Nik| 26|    2|
+----+---+-----+



In [10]:
df1.groupBy(df1.columns).count().where( "count > 1" ).drop('count').show()

+----+---+
|Name|Age|
+----+---+
| Akh| 31|
|Sony| 35|
| Nik| 26|
+----+---+



Window Functions - row_number() :

In [11]:
from pyspark.sql.functions import col,row_number
from pyspark.sql.window import Window

In [12]:
win = Window.partitionBy("Name").orderBy(col("Age").desc())

In [15]:
df1.withColumn("Rank",row_number().over(win)).show()

+-----+---+----+
| Name|Age|Rank|
+-----+---+----+
|  Akh| 31|   1|
|  Akh| 31|   2|
|  Akh| 31|   3|
| Demo| 21|   1|
|Nibba| 16|   1|
|Nibbi| 17|   1|
|  Nik| 26|   1|
|  Nik| 26|   2|
| Sony| 35|   1|
| Sony| 35|   2|
| Sony| 35|   3|
+-----+---+----+



In [17]:
df1.withColumn("Rank",row_number().over(win)).where('Rank>1').show()

+----+---+----+
|Name|Age|Rank|
+----+---+----+
| Akh| 31|   2|
| Akh| 31|   3|
| Nik| 26|   2|
|Sony| 35|   2|
|Sony| 35|   3|
+----+---+----+



In [18]:
df1.withColumn("Rank",row_number().over(win)).where('Rank>1').drop("Rank").distinct().show()

+----+---+
|Name|Age|
+----+---+
| Akh| 31|
| Nik| 26|
|Sony| 35|
+----+---+

